# Fase 7: Interfaz Web de Usuario

### Objetivo:
El objetivo de esta fase es desarrollar una interfaz de usuario intuitiva y funcional que permita a los usuarios realizar consultas de búsqueda en el motor de búsqueda implementado. La interfaz de usuario debe proporcionar un medio para ingresar consultas, seleccionar el modelo de búsqueda deseado y mostrar los resultados de manera clara y organizada.

### Descripción:
Durante esta etapa, utilizamos el framework Flask para desarrollar la interfaz de usuario del motor de búsqueda. Creamos un formulario HTML que permite a los usuarios ingresar consultas y seleccionar el modelo de búsqueda (TF-IDF con Coseno o BoW con Jaccard) a utilizar. Al enviar el formulario, la consulta y el modelo seleccionado se envían al servidor Flask, que procesa la consulta y devuelve los resultados correspondientes.

### Pasos para la interfaz de usuario:
1. **Creación del archivo HTML:**
    - El archivo HTML proporciona la estructura y los elementos de la interfaz de usuario, incluido el formulario de búsqueda y el contenedor para mostrar los resultados de la búsqueda. También incluye referencias a la hoja de estilos, a las bibliotecas Bootstrap y jQuery para mejorar el estilo y la funcionalidad de la interfaz.
    - En el script de JavaScript se envía una consulta, se activa un evento que procesa la solicitud. Primero, se obtienen la consulta y el modelo seleccionado del formulario. Luego, se envía una solicitud al servidor Flask para procesar la consulta utilizando el modelo elegido. Mientras se espera la respuesta del servidor, se muestra un indicador de carga para mejorar la experiencia del usuario. Una vez que se recibe la respuesta, se maneja para mostrar los resultados en la página. Cada resultado es interactivo y permite al usuario abrir el documento correspondiente en una nueva ventana al hacer clic en él.

2. **Configuración del Servidor Flask (app.py):**
    - El servidor Flask se encarga de manejar las solicitudes del cliente y procesar la consulta de búsqueda. Define dos rutas principales: '/' para renderizar el formulario de búsqueda y '/process' para procesar la consulta enviada por el usuario. Se utiliza el método POST para enviar datos de consulta al servidor.

3. **Manejo de Consultas (app.py):**
    - Cuando el usuario envía una consulta de búsqueda a través del formulario, el servidor Flask procesa la consulta y el modelo seleccionado (TF-IDF  o BoW). Se valida la consulta y el modelo antes de pasarlos al motor de búsqueda para recuperar los resultados correspondientes.

4. **Renderización de Resultados (app.py y index.html):**
    - Una vez que se procesa la consulta, los resultados se envían de vuelta al cliente y se muestran en la interfaz de usuario. Los documentos recuperados se muestran en una lista dentro del contenedor de resultados. Se proporciona la opción de hacer clic en cada resultado para ver el documento completo.

5. **Manejo de Errores (app.py y index.html):**
    - Se implementa la gestión de errores en el servidor Flask para manejar situaciones como consultas vacías o modelos de búsqueda no válidos. Los mensajes de error se muestran en la interfaz de usuario para informar al usuario sobre cualquier problema que surja durante el proceso de búsqueda.

In [1]:
# Instalar Flask
!pip install Flask
!pip install gunicorn
!pip install waitress

In [1]:
from ipynb.fs.full.SRI_Reuters_converted import buscar_documentos

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Ãƒndice invertido guardado en inverted_index.txt
Busqueda para la query: lin-oil
Resultados ordenados (nombre y distancia):
Nombre: 6.txt, Distancia: 0.06864310990282788

Nombres de documentos ordenados:
6.txt
Busqueda para la query: lin-oil
Resultados ordenados (nombre y distancia):
Nombre: 6.txt, Distancia: 0.15617376188860607

Nombres de documentos ordenados:
6.txt


In [2]:
%%writefile wsgi.py
from busqueda import app as application

if __name__ == "__main__":
    application.run()

Overwriting wsgi.py


In [3]:
from flask import Flask, request, jsonify, render_template
from waitress import serve

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/process', methods=['POST'])
def process():
    try:
        data = request.json
        query = data.get('query')
        model = data.get('model')

        if not query:
            return jsonify({'error': 'No query provided'}), 400

        if model == "0":  # TF-IDF con Coseno
            vectorizer_type = 'TF-IDF'
        elif model == "1":  # BoW con Jaccard
            vectorizer_type = 'BoW'
        else:
            return jsonify({'error': 'Invalid model selection'}), 400

        resultados_ordenados, nombres_ordenados = buscar_documentos(query, vectorizer_type)
        return jsonify({'result': nombres_ordenados})

    except ValueError as e:
        print(f"ValueError: {str(e)}")
        return jsonify({'error': str(e)}), 400
    except Exception as e:
        print(f"Exception: {str(e)}")
        return jsonify({'error': 'Internal Server Error', 'details': str(e)}), 500

if __name__ == '__main__' and '__file__' not in globals():
    host = '127.0.0.1'
    port = 5000
    print(f"Servidor en ejecución en http://{host}:{port}")
    serve(app, host=host, port=port)


Servidor en ejecución en http://127.0.0.1:5000
